### Try to correlate the Jaccard Similarity with the Synergy Score ###

- Need to figure out whether to break down based on the cell line

In [1]:
# Import everything needed
from toxicity_ranking import *
import numpy as np

Get the drugcomb df and investigate whether there's a

In [3]:
drugcomb_df_all = get_drug_comb_data()
drugcomb_df = get_drug_comb_data(bliss=True, loewe=True, hsa=True, zip=True)
sider_cid_to_drugs_df, sider_all_side_effects_df = get_sider_data()

#Filter data and get jaccard similarity
filtered_drug_comb_data, common_drugs, unique_drug_pairs = filter_drug_comb_data(drugcomb_df, sider_cid_to_drugs_df)
drug_pair_to_jaccard, drug_pair_to_side_effects = drug_pair_to_jaccard_similarity(unique_drug_pairs, sider_cid_to_drugs_df, sider_all_side_effects_df)

# Rank the drug pairs by Jaccard similarity
ranked_drug_pairs = rank_drug_pairs(drug_pair_to_jaccard)

/Users/alexwong/Research/toxicity-cancer-drug-combination/toxicity_ranking.py:13: DtypeWarning: Columns (2,7,19,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  drugcomb_df = pd.read_csv('data/DrugComb/drugcomb_summary_v_1_5.csv', sep=',', index_col=False)


Original shape of drugcomb data:  (1432351, 26)
Final shape of filtered drugcomb data:  (722344, 26)


/Users/alexwong/Research/toxicity-cancer-drug-combination/toxicity_ranking.py:13: DtypeWarning: Columns (2,7,19,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  drugcomb_df = pd.read_csv('data/DrugComb/drugcomb_summary_v_1_5.csv', sep=',', index_col=False)


Original shape of drugcomb data:  (1432351, 26)
Final shape of filtered drugcomb data:  (123882, 26)
Original drugcomb data shape:  (123882, 26)
Number of drugs in common between drugcomb and sider [lowercase enforced]:  401
Filtered drugcomb data shape for both drugs being present in sider:  (20824, 26)
Number of unique drug pairs:  6552


In [5]:
# Make sure all of the synergy score options are numeric, drop the row otherwise
drugcomb_df_all['synergy_bliss'] = pd.to_numeric(drugcomb_df_all['synergy_bliss'], errors='coerce')
drugcomb_df_all['synergy_loewe'] = pd.to_numeric(drugcomb_df_all['synergy_loewe'], errors='coerce')
drugcomb_df_all['synergy_hsa'] = pd.to_numeric(drugcomb_df_all['synergy_hsa'], errors='coerce')
drugcomb_df_all['synergy_zip'] = pd.to_numeric(drugcomb_df_all['synergy_zip'], errors='coerce')
drugcomb_df_all = drugcomb_df_all.dropna(subset=['synergy_bliss', 'synergy_loewe', 'synergy_hsa', 'synergy_zip'])

drugcomb_df_all

,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,ic50_row,ic50_col,...,S_mean,S_max,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,drug_row_clinical_phase,drug_col_clinical_phase,drug_row_target_name,drug_col_target_name
19037,452434,lonidamine,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,7.604007e-02,4.496970,...,39.0610,24.9095,0.409635,-11.702283,-1.252282,-1.212833,3,0,Beta-lactamase AmpC,Focal adhesion kinase 1; Protein tyrosine kina...
19038,452435,ethyl bromopyruvate,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,4.000000e+00,3.605996,...,23.6110,12.3940,-3.977257,-16.185120,-12.449563,-11.356646,0,0,\N,Focal adhesion kinase 1; Protein tyrosine kina...
19039,452436,tranilast (trans-),717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,1.437738e-02,2.802769,...,16.2610,5.5935,0.117053,-16.588246,-4.052052,-1.243804,0,0,\N,Focal adhesion kinase 1; Protein tyrosine kina...
19040,452437,lenalidomide,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,7.631303e-09,3.926323,...,27.8965,13.9555,2.606357,-10.877569,0.456392,3.936726,4,0,Cereblon/Aiolos; Cereblon/Ikaros; CRL4(CRBN) E...,Focal adhesion kinase 1; Protein tyrosine kina...
19041,452438,pomalidomide,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,2.074997e+00,3.428020,...,29.2920,20.9890,2.674166,-1.901326,3.045460,5.292969,4,0,Cereblon/Aiolos; Cereblon/Ikaros; CRL4(CRBN) E...,Focal adhesion kinase 1; Protein tyrosine kina...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431628,1197317,azd4547,carfilzomib,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,3.000000e+00,0.010080,...,30.5715,12.1875,-3.500767,-1.732535,1.567371,-3.539042,3,4,Fibroblast growth factor receptor 3; Fibroblas...,Proteasome subunit beta type-8; 26S proteasome...
1431629,1197428,azd4547,cediranib,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,1.168609e+00,0.871137,...,29.1820,28.4455,1.971150,1.867163,5.251662,4.061791,3,3,Fibroblast growth factor receptor 3; Fibroblas...,Vascular endothelial growth factor receptor 2;...
1431630,1197492,azd4547,az13401366,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,1.802866e+00,1.195275,...,48.2525,44.9730,19.276304,11.575918,16.003936,21.509903,3,\N,Fibroblast growth factor receptor 3; Fibroblas...,\N
1431631,1198955,azd5582,tnf-alpha,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,6.355483e-02,0.000388,...,6.4540,1.6725,-3.158395,3.923324,4.500456,-3.339732,0,\N,Inhibitor of apoptosis protein 3; Baculoviral ...,\N


In [7]:
# Are there multiple synergy scores per unique drug pair

drug_pair_to_bliss_synergy_score_count = {}
drug_pair_to_bliss_cell_line_count = {}
# Try Bliss synergy score only

for drug_pair in unique_drug_pairs:
    # Locate in drugcomb_df
    drug1, drug2 = drug_pair
    drug1_row_present = drugcomb_df_all['drug_row'] == drug1
    drug1_col_present = drugcomb_df_all['drug_col'] == drug1
    drug1_present = drug1_row_present | drug1_col_present
    drug2_row_present = drugcomb_df_all['drug_row'] == drug2
    drug2_col_present = drugcomb_df_all['drug_col'] == drug2
    drug2_present = drug2_row_present | drug2_col_present
    both_drugs_present = drugcomb_df_all[drug1_present & drug2_present]

    # How many Bliss synergy scores are there for this drug pair? Store in drug_pair_to_bliss_synergy_score_count
    drug_pair_to_bliss_synergy_score_count[drug_pair] = len(both_drugs_present['synergy_bliss'])
    drug_pair_to_bliss_cell_line_count[drug_pair] = len(both_drugs_present['cell_line_name'].unique())

drug_pair_to_bliss_synergy_score_count
    


{('orlistat', 'carfilzomib'): 1,
 ('palonosetron', 'imipramine'): 1,
 ('pentamidine', 'nitazoxanide'): 1,
 ('palonosetron', 'trihexyphenidyl'): 1,
 ('aprepitant', 'olmesartan'): 1,
 ('palonosetron', 'zaleplon'): 1,
 ('maraviroc', 'ramelteon'): 1,
 ('leflunomide', 'carbachol'): 1,
 ('cabazitaxel', 'ruxolitinib'): 118,
 ('docetaxel', 'mitotane'): 59,
 ('probenecid', 'maraviroc'): 1,
 ('dacarbazine', 'amifostine'): 60,
 ('levonorgestrel', 'irinotecan'): 1,
 ('rufinamide', 'eltrombopag'): 1,
 ('amantadine', 'mephenytoin'): 1,
 ('azithromycin', 'pentostatin'): 1,
 ('palonosetron', 'albendazole'): 1,
 ('6-mercaptopurine', 'prednisolone'): 1,
 ('mitotane', 'anastrozole'): 61,
 ('orlistat', 'allopurinol'): 1,
 ('dantrolene', 'vinblastine'): 1,
 ('dacarbazine', 'ixabepilone'): 59,
 ('gefitinib', 'methyclothiazide'): 1,
 ('vinorelbine', 'vorinostat'): 156,
 ('temozolomide', 'paromomycin'): 1,
 ('cabazitaxel', 'bleomycin'): 59,
 ('clindamycin', 'temozolomide'): 1,
 ('axitinib', 'nilotinib'): 60,


In [10]:
print("Unique cell lines for vinorelbine and vorinostat: ", drug_pair_to_bliss_cell_line_count[('vinorelbine', 'vorinostat')])

#what are the actual synergy scores for vinorelbine and vorinostat
drug1 = 'vinorelbine'
drug2 = 'vorinostat'
drug1_row_present = drugcomb_df_all['drug_row'] == drug1
drug1_col_present = drugcomb_df_all['drug_col'] == drug1
drug1_present = drug1_row_present | drug1_col_present
drug2_row_present = drugcomb_df_all['drug_row'] == drug2
drug2_col_present = drugcomb_df_all['drug_col'] == drug2
drug2_present = drug2_row_present | drug2_col_present
both_drugs_present = drugcomb_df_all[drug1_present & drug2_present]

bliss_syn_scores = both_drugs_present['synergy_bliss'].values

print("Vinorelbine and vorinostat bliss synergy scores include: ", set(bliss_syn_scores))

# How many unique bliss synergy scores are there? do they match the number of cell lines?
len_bliss = len(set(bliss_syn_scores))
print("Number of unique bliss synergy scores: ", len_bliss)


Unique cell lines for vinorelbine and vorinostat:  39
Vinorelbine and vorinostat bliss synergy scores include:  {0.627623687221109, 1.02694164666446, 2.82280875153977, 3.85669183103309, 4.71530875153977, 5.37425414666447, 3.44105875153977, 4.51430875153977, 5.80651995420885, 9.41994164666446, 4.36459991313254, 11.9903949542089, 12.1457874131325, 9.02322491313254, 1.06812832519703, 8.07556118722111, 14.0884801290487, 8.98650147470252, 9.96289837225686, 10.5196260119108, 13.8181885119108, 21.0192301290487, 4.53907519932843, 23.9139176290487, -2.42804254579115, 5.62166241313254, 5.91487601191082, -7.77833369862251, -5.52879829739335, -1.23964260322851, 2.77110432359487, 1.25260778239953, -1.51207569486614, -1.43194997010358, -0.0562300457911484, -0.201992224380058, 1.14975777561994, 1.58499868722111, 1.21806836859448, 4.4640728973747, -2.0967886870174, -7.97143962540464, -7.29934551499643, 7.6947926290487, -5.7051718969833, 2.66992899009921, -4.41708030402828, -0.150199970103577, -0.30689

Get the average synergy score for each unique drug pair and then plot an xy graph of jaccard similarity to synergy score, calculate pearson correlation

In [11]:
drug_pair_to_average_bliss_ss = {}
drug_pair_to_average_loewe_ss = {}
drug_pair_to_average_hsa_ss = {}
drug_pair_to_average_zip_ss = {}

for drug_pair in unique_drug_pairs:
    # Locate in drugcomb_df
    drug1, drug2 = drug_pair
    drug1_row_present = drugcomb_df_all['drug_row'] == drug1
    drug1_col_present = drugcomb_df_all['drug_col'] == drug1
    drug1_present = drug1_row_present | drug1_col_present
    drug2_row_present = drugcomb_df_all['drug_row'] == drug2
    drug2_col_present = drugcomb_df_all['drug_col'] == drug2
    drug2_present = drug2_row_present | drug2_col_present
    both_drugs_present = drugcomb_df_all[drug1_present & drug2_present]

    # Calculate the average Bliss synergy score for this drug pair
    drug_pair_to_average_bliss_ss[drug_pair] = np.mean(both_drugs_present['synergy_bliss'])
    drug_pair_to_average_loewe_ss[drug_pair] = np.mean(both_drugs_present['synergy_loewe'])
    drug_pair_to_average_hsa_ss[drug_pair] = np.mean(both_drugs_present['synergy_hsa'])
    drug_pair_to_average_zip_ss[drug_pair] = np.mean(both_drugs_present['synergy_zip'])



In [13]:
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Sort out the data for plotting
bliss_ss = [bliss_score for drug_pair in unique_drug_pairs for bliss_score in [drug_pair_to_average_bliss_ss[drug_pair]]]
loewe_ss = [loewe_score for drug_pair in unique_drug_pairs for loewe_score in [drug_pair_to_average_loewe_ss[drug_pair]]]
hsa_ss = [hsa_score for drug_pair in unique_drug_pairs for hsa_score in [drug_pair_to_average_hsa_ss[drug_pair]]]
zip_ss = [zip_score for drug_pair in unique_drug_pairs for zip_score in [drug_pair_to_average_zip_ss[drug_pair]]]
jaccard_sim = [jaccard_score for drug_pair in unique_drug_pairs for jaccard_score in [drug_pair_to_jaccard[drug_pair]]]

# correlation between jaccard similarity and bliss synergy score
bliss_pearson_coef, bliss_p_value = pearsonr(jaccard_sim, bliss_ss)
print("Pearson correlation coefficient for JS and Bliss: ", bliss_pearson_coef)
print("Pearson pval for JS and Bliss", bliss_p_value)
loewe_pearson_coef, loewe_p_value = pearsonr(jaccard_sim, loewe_ss)
print("Pearson correlation coefficient for JS and Loewe: ", loewe_pearson_coef)
print("Pearson pval for JS and Loewe", loewe_p_value)
hsa_pearson_coef, hsa_p_value = pearsonr(jaccard_sim, hsa_ss)
print("Pearson correlation coefficient for JS and HSA: ", hsa_pearson_coef)
print("Pearson pval for JS and HSA", hsa_p_value)
zip_pearson_coef, zip_p_value = pearsonr(jaccard_sim, zip_ss)
print("Pearson correlation coefficient for JS and ZIP: ", zip_pearson_coef)
print("Pearson pval for JS and ZIP", zip_p_value)

# Plot the data
plt.scatter(jaccard_sim, bliss_ss)
plt.xlabel('Jaccard similarity')
plt.ylabel('Average Bliss synergy score')
plt.title('Jaccard similarity vs. Bliss synergy score')
plt.savefig('results/jaccard_vs_bliss.png')
plt.close()

plt.scatter(jaccard_sim, loewe_ss)
plt.xlabel('Jaccard similarity')
plt.ylabel('Average Loewe synergy score')
plt.title('Jaccard similarity vs. Loewe synergy score')
plt.savefig('results/jaccard_vs_loewe.png')
plt.close()

plt.scatter(jaccard_sim, hsa_ss)
plt.xlabel('Jaccard similarity')
plt.ylabel('Average HSA synergy score')
plt.title('Jaccard similarity vs. HSA synergy score')
plt.savefig('results/jaccard_vs_hsa.png')
plt.close()

plt.scatter(jaccard_sim, zip_ss)
plt.xlabel('Jaccard similarity')
plt.ylabel('Average ZIP synergy score')
plt.title('Jaccard similarity vs. ZIP synergy score')
plt.savefig('results/jaccard_vs_zip.png')
plt.close()

Pearson correlation coefficient for JS and Bliss:  -0.03288232643316114
Pearson pval for JS and Bliss 0.0077711820353147726
Pearson correlation coefficient for JS and Loewe:  -0.10103373111313242
Pearson pval for JS and Loewe 2.4535704755557756e-16
Pearson correlation coefficient for JS and HSA:  -0.07600954690336152
Pearson pval for JS and HSA 7.261274318671033e-10
Pearson correlation coefficient for JS and ZIP:  -0.028068544607751905
Pearson pval for JS and ZIP 0.023086128173173237


In [15]:
# Double check that the bliss synergy score and the sorting is correct
drug_pair_0 = list(unique_drug_pairs)[0]
print("Checking first drug pair: ", drug_pair_0)
print("Jaccard similarity: ", drug_pair_to_jaccard[drug_pair_0])
print("X value used for plotting: ", jaccard_sim[0])
print("Average Bliss synergy score: ", drug_pair_to_average_bliss_ss[drug_pair_0])
print("Y value used for plotting: ", bliss_ss[0])
print("Average Loewe synergy score: ", drug_pair_to_average_loewe_ss[drug_pair_0])
print("Y value used for plotting: ", loewe_ss[0])
print("Average HSA synergy score: ", drug_pair_to_average_hsa_ss[drug_pair_0])
print("Y value used for plotting: ", hsa_ss[0])
print("Average ZIP synergy score: ", drug_pair_to_average_zip_ss[drug_pair_0])
print("Y value used for plotting: ", zip_ss[0])


Checking first drug pair:  ('orlistat', 'carfilzomib')
Jaccard similarity:  0.102803738317757
X value used for plotting:  0.102803738317757
Average Bliss synergy score:  14.6084638545148
Y value used for plotting:  14.6084638545148
Average Loewe synergy score:  11.4184961171588
Y value used for plotting:  11.4184961171588
Average HSA synergy score:  11.6041890533173
Y value used for plotting:  11.6041890533173
Average ZIP synergy score:  12.305516469874
Y value used for plotting:  12.305516469874
